# **[WIP] BipedalWalker - Gaussian PPO (Experimental)**

The implementation of PPO with gaussian critic for BipedalWalker v3.



## Initialization

In [1]:
%matplotlib inline
from IPython import display
import matplotlib.pyplot as plt

In [ ]:
from sim.GymTrainer import GymTrainer
from framework.GaussianPPO import GaussianPPOAgent

import torch
trainDevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Create the simulation environment
sim = GymTrainer(
    'BipedalWalker-v3', 
    evalDevice="cpu", 
    trainDevice=trainDevice, 
    render_mode='rgb_array', 
    envNum=1,
    hardcore=False,
    batchSize=8000,
    maxEpisode=6000,
    maxStep=100000,
    seed=555)

In [ ]:
# Create the agent
agent = GaussianPPOAgent(
    actionNum=sim.actionSize(), 
    stateNum=sim.stateSize(), 
    gamma=0.99, 
    lamda=0.95,
    eps=0.2,
    rwShaper=lambda rwds: torch.clamp(rwds, min = -1.0),
    entropyBeta=0.0002,
    criticEntropyBeta = 0.0001,
    entropySupressCoef = 0.5,
    memorySize=8000,
    batchSize=2000,
    trainEpoch=70,
    policyLR=1e-4, 
    criticLR=1e-4,
    layerActor=[64] ,
    layerCritic=[64])

In [ ]:
name, writer = sim.makeSummaryWriter(agent)

## Training

In [ ]:
sim.train(agent, writer)

In [ ]:
sim.test(
    agent = agent, 
    episode = 40, 
    maxStep = 10000, 
    renderStep = 5,
    writer = None)